In [1]:
!pip install pyodbc sqlalchemy pymysql pandas


     ---------------------------------------- 68.9/68.9 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ---------------------------------------- 45.0/45.0 kB ? eta 0:00:00
     --------------------------------------- 11.1/11.1 MB 11.1 MB/s eta 0:00:00
     -------------------------------------- 296.6/296.6 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 347.8/347.8 kB 7.3 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 kB 7.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pyodbc

# Update with your SQL Server details
server = 'DESKTOP-D933C06\SQLEXPRESS'
database = 'connection'
username = 'sa'
password = 'tharun'

conn_sqlserver = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

cursor_sqlserver = conn_sqlserver.cursor()


In [10]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# Connection details
mysql_user = 'root'
mysql_password = 'Tharun@19072003'
mysql_host = 'localhost'
mysql_port = '3306'
mysql_db = 'connection'

# URL encode the password
encoded_password = quote_plus(mysql_password)

# Create connection string
mysql_conn_str = f'mysql+pymysql://{mysql_user}:{encoded_password}@{mysql_host}:{mysql_port}/{mysql_db}'
engine_mysql = create_engine(mysql_conn_str)

# Test connection
with engine_mysql.connect() as conn:
    result = conn.execute(text("SELECT VERSION()"))  # ✅ wrap SQL in text()
    print(f"MySQL Version: {result.fetchone()[0]}")


MySQL Version: 8.0.42


In [11]:
# Create SQL Server table if it doesn't exist
cursor_sqlserver.execute("""
IF NOT EXISTS (
    SELECT * FROM sysobjects WHERE name='source_table' AND xtype='U'
)
CREATE TABLE source_table (
    id INT PRIMARY KEY,
    name VARCHAR(100),
    salary FLOAT
)
""")
conn_sqlserver.commit()
print("SQL Server table 'source_table' created.")


SQL Server table 'source_table' created.


In [ ]:
# Sample data to insert
#data = [
#    (1, 'Rohini', 90000),
 #   (2, 'Monika', 89000),
  #  (3, 'Pavani', 70000),
   # (4, 'Naveena', 1000)
#]

# Loop through each row and delete existing, then insert
#for row in data:
  #  id_, name, salary = row

    # Delete if exists
 #   delete_query = "DELETE FROM source_table WHERE id = ?"
 #   cursor_sqlserver.execute(delete_query, (id_,))

    # Insert new data
   # insert_query = "INSERT INTO source_table (id, name, salary) VALUES (?, ?, ?)"
  #  cursor_sqlserver.execute(insert_query, row)

# Commit the transaction
#conn_sqlserver.commit()
#print("Existing rows deleted (if any) and new data inserted successfully.")


Existing rows deleted (if any) and new data inserted successfully.


In [12]:
# Insert sample data
insert_query = "INSERT INTO source_table (id, name, salary) VALUES (?, ?, ?)"
data = [
    (1, 'Rohini', 90000),
    (2, 'Monika', 89000),
    (3, 'Pavani', 70000),
    (4,'Naveena',1000)
]

cursor_sqlserver.executemany(insert_query, data)
conn_sqlserver.commit()
print("Sample data inserted into SQL Server 'source_table'.")


Sample data inserted into SQL Server 'source_table'.


In [13]:
import pandas as pd

# Replace with your actual table name
sql_query = "SELECT * FROM source_table"
df = pd.read_sql(sql_query, conn_sqlserver)

# Optional transformation
#df['new_column'] = df['salary'].apply(lambda x: x * 2)  # if 'salary' exists

# Load into MySQL
df.to_sql('target_table', con=engine_mysql, if_exists='replace', index=False)


C:\Users\Rohini\AppData\Local\Temp\ipykernel_7688\3154384991.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_sqlserver)


4